In [ ]:
# ==================================================================================================================
# ===========================================> Instalación librerías <==============================================
# ==================================================================================================================

#%pip install pandas    # NO HACE FALTA INSTALAR
#%pip install numpy     # NO HACE FALTA INSTALAR
#%pip install pickle    # NO HACE FALTA INSTALAR
#%pip install sklearn   # NO HACE FALTA INSTALAR

#%pip install scikit-learn          # DESCOMENTAR LA PRIMERA VEZ
#%pip install category_encoders     # DESCOMENTAR LA PRIMERA VEZ
#%pip install lightgbm              # DESCOMENTAR LA PRIMERA VEZ
#%pip install matplotlib            # DESCOMENTAR LA PRIMERA VEZ
#%pip install seaborn               # DESCOMENTAR LA PRIMERA VEZ



In [ ]:
# ==================================================================================================================
# ===========================================> Carga librerías <====================================================
# ==================================================================================================================

import pandas as pd
import numpy as np
import pickle

# Transformación de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
import category_encoders as ce

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

# Seleccion de variables y tuning de hiperparámetros
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Métricas para evaluar un modelo de clasificación
from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_curve, roc_auc_score, average_precision_score, confusion_matrix

# Librerías para visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns

# Para que no se corten el listado de filas y columnas al ejecutar instrucciones
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# ==================================================================================================================
# ===========================================> Carga Datos <========================================================
# ==================================================================================================================

df = pd.read_csv('../data/train.csv', low_memory=False)
df = df.sample(10000)

# =====================> EDA

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración número de columnas <=====================================
# ==================================================================================================================

len(df.columns)

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración estadisticas basicas col numericas <=====================
# ==================================================================================================================

df.describe()

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración estadisticas basicas col objectos <=====================
# ==================================================================================================================

df.describe(include=object)

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración 20 primeras filas <======================================
# ==================================================================================================================

df.head(20)

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración tipos <==================================================
# ==================================================================================================================

df.info()

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración nulos <==================================================
# ==================================================================================================================

df.isna().sum().sort_values(ascending=False)

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración valores y estadísticas <=================================
# ==================================================================================================================

stats = []
# Unique Values         col 
# Unique Values         df[col].nunique() 
# Unique Values %       (df[col].nunique() / df[col].count()) * 100
# Missing Values %      df[col].isnull().sum() * 100 / df.shape[0]
# Biggest Category %    df[col].value_counts(normalize=True, dropna=False).values[0] * 100
# Type                  df[col].dtype

for col in df.columns:
    stats.append((col, df[col].nunique(), (df[col].nunique() / df[col].count()) * 100, df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    
stats_df = pd.DataFrame(stats, columns=['Feature', 'Unique Values', 'Unique Values %', 'Missing Values %', 'Biggest Category %', 'Type'])
stats_df.sort_values('Missing Values %', ascending=False)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar valores nulos <=============================================
# ==================================================================================================================

drop_cols_min = 0.6
drop_cols = []

for c in df.columns:
    if df[c].isna().sum() / len(df) > drop_cols_min:
        drop_cols.append(c)
        
for c in drop_cols:
    df = df.drop(c, axis=1)

print(drop_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar valores muy balanceados <===================================
# ==================================================================================================================

drop_cols_min_big_cat = 70
drop_cols = []

for c in df.columns:
    if df[c].value_counts(normalize=True, dropna=False).values[0] * 100 >= drop_cols_min_big_cat:
        drop_cols.append(c)
        
for c in drop_cols:
    df = df.drop(c, axis=1)

print(drop_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar valores muy balanceados <===================================
# ==================================================================================================================

drop_cols_min_unique = 2
drop_cols_min_big_cat = 60
drop_cols = []

for c in df.columns:
    if df[c].nunique() <= drop_cols_min_unique and df[c].value_counts(normalize=True, dropna=False).values[0] * 100 >= drop_cols_min_big_cat:
        drop_cols.append(c)
        
for c in drop_cols:
    df = df.drop(c, axis=1)

print(drop_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar imputar nulos <=============================================
# ==================================================================================================================

# Se imputa todos los valores nulos como la moda, ya que en su mayoria son categoricos o identificadores

imputed_min_unique = 100
imputed_cols = []

for c in df.columns:
    if df[c].isnull().any() and ((df[c].nunique() / df[c].count()) * 100) <= imputed_min_unique:
        imputed_cols.append(c)
        
for c in imputed_cols:
    df[c] = df[c].fillna(df[c].mode()[0])

print(imputed_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 3 partes <=============================
# ==================================================================================================================

mask_cols = []

for c in df.columns:
    if df[c].notnull().all() and df[c].astype(str).apply(lambda x: x.count('.') == 2).all():
        mask_cols.append(c)

for c in mask_cols:
    df[[c + "_1", c + "_2", c + "_3"]] = df[c].str.split(".", expand=True)

for c in mask_cols:
    df = df.drop(c, axis=1)

print(mask_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 4 partes <=============================
# ==================================================================================================================

mask_cols = []

for c in df.columns:
    if df[c].notnull().all() and df[c].astype(str).apply(lambda x: x.count('.') == 3).all():
        mask_cols.append(c)

for c in mask_cols:
    df[[c + "_1", c + "_2", c + "_3", c + "_4"]] = df[c].str.split(".", expand=True)

for c in mask_cols:
    df = df.drop(c, axis=1)

print(mask_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 5 partes <=============================
# ==================================================================================================================

mask_cols = []

for c in df.columns:
    if df[c].notnull().all() and df[c].astype(str).apply(lambda x: x.count('.') == 4).all():
        mask_cols.append(c)

for c in mask_cols:
    df[[c + "_1", c + "_2", c + "_3", c + "_4", c + "_5"]] = df[c].str.split(".", expand=True)

for c in mask_cols:
    df = df.drop(c, axis=1)

print(mask_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 6 partes <=============================
# ==================================================================================================================

mask_cols = []

for c in df.columns:
    if df[c].notnull().all() and df[c].astype(str).apply(lambda x: x.count('.') == 5).all():
        mask_cols.append(c)

for c in mask_cols:
    df[[c + "_1", c + "_2", c + "_3", c + "_4", c + "_5", c + "_6"]] = df[c].str.split(".", expand=True)

for c in mask_cols:
    df = df.drop(c, axis=1)

print(mask_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar Unique Tipos Cat <==========================================
# ==================================================================================================================

typed_min_unique = 10
typed_cols = []

for c in df.columns:
    if ((df[c].nunique() / df[c].count()) * 100) <= typed_min_unique:
        typed_cols.append(c)
        
for c in typed_cols:
    df[c] = df[c].astype("category")

print(typed_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar Object Tipos Cat <==========================================
# ==================================================================================================================

typed_cols = []

for c in df.columns:
    if df[c].dtype == "object":
        typed_cols.append(c)
        
for c in typed_cols:
   df[c] = df[c].astype("category")

print(typed_cols)